In [14]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [15]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-10-30 05:07:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-10-30 05:07:58 (225 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [16]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [17]:
# Preprocess data to filter users and books with minimum ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge with book titles
df = pd.merge(df_ratings, df_books, on='isbn')

# Create a pivot table for user-item interactions
book_user_matrix = df.pivot(index='title', columns='user', values='rating').fillna(0)
book_user_sparse_matrix = csr_matrix(book_user_matrix.values)

# Train the KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
# Cell 5
# Define the get_recommends function
def get_recommends(book=""):
    # Debugging: Check if book is in the matrix
    if book not in book_user_matrix.index:
        print(f"Book '{book}' not found in dataset.")
        return [book, []]

    # Find the index of the book
    book_index = book_user_matrix.index.get_loc(book)

    # Run KNN to find nearest neighbors
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    # Create the recommendation list
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_book = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommended_books.append([recommended_book, distance])

    # Check if recommendations were found
    if len(recommended_books) == 0:
        print(f"No recommendations found for '{book}'.")

    return [book, recommended_books[:5]]

In [28]:
# Cell 6: Define recommendation function
def get_recommends(book=""):
    # Check if book exists in matrix
    if book not in book_user_matrix.index:
        return [book, []]

    # Get book index
    book_index = book_user_matrix.index.get_loc(book)

    # Get nearest neighbors
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    # Compile recommendation list
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_book = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommended_books.append([recommended_book, distance])

    return [book, recommended_books[:5]]

# Test the function
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

# Cell 7: Test recommendation function with additional feedback
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

    # Check if primary book matches
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        print(f"Primary book does not match: {recommends[0]}")
        test_pass = False

    # Define expected recommendations and distances
    expected_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    expected_distances = [0.8, 0.77, 0.77, 0.77]

    # Compare recommendations and distances
    for i in range(4):
        if i >= len(recommends[1]):
            print(f"Recommendation {i + 1} missing.")
            test_pass = False
            continue

        recommended_title, recommended_distance = recommends[1][i]

        if recommended_title not in expected_books:
            print(f"Unexpected recommended book: {recommended_title}")
            test_pass = False

        if abs(recommended_distance - expected_distances[i]) >= 0.05:
            print(f"Distance mismatch for {recommended_title}: got {recommended_distance}, expected {expected_distances[i]}")
            test_pass = False

    # Final result message
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

# Run the test function
test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864], ["The Pilot's Wife : A Novel", 0.81926787], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.82366824], ['Bel Canto: A Novel', 0.8247875]]]
Unexpected recommended book: The Lovely Bones: A Novel
Distance mismatch for The Lovely Bones: A Novel: got 0.7234864234924316, expected 0.8
Unexpected recommended book: The Pilot's Wife : A Novel
Unexpected recommended book: The Joy Luck Club
Unexpected recommended book: The Notebook
Distance mismatch for The Notebook: got 0.8236682415008545, expected 0.77
You haven't passed yet. Keep trying!
